In [1]:
# https://stackoverflow.com/questions/45741878/using-binary-crossentropy-loss-in-keras-tensorflow-backend
# https://www.machinecurve.com/index.php/2019/10/22/how-to-use-binary-categorical-crossentropy-with-keras/

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.model_selection import train_test_split
from keras.callbacks import CSVLogger
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

%load_ext tensorboard
import datetime

!rm -rf ./logs/LSTM/

tf.config.experimental.list_physical_devices('GPU')

# RNN based Models (RNN & LSTM & GRU) using tensorflow keras

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
#normalize X values to help model converge
def normalize(data):
    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    data = (data - mean) / std
    return data

In [3]:
def convert_3D(matrix):
    return np.expand_dims(matrix, -1)

In [4]:
#retrieve and format data - into labels and examples from the dataset
def features_and_labels(filename_a, filename_b):
    with open(filename_b, 'rb') as f:
        labels = np.load(f).ravel()
    
    data = pd.read_pickle(filename_a)
    #https://stackoverflow.com/questions/22798934/pandas-long-to-wide-reshape-by-two-variables
    data_x = data.pivot(index='id', columns='time', values='FLUX')
    data_x = data_x.to_numpy()
    data_x = convert_3D(normalize(data_x))
    return data_x, labels

In [5]:
training_data_x, training_data_y = features_and_labels("../Kepler-Train.pkl", "../Labels-Train.npy")
testing_data_x, testing_data_y = features_and_labels("../Kepler-Test.pkl", "../Labels-Test.npy")
validation_data_x, validation_data_y = features_and_labels("../Kepler-Val.pkl", "../Labels-Val.npy")

In [6]:
LSTMmodel = Sequential([
    LSTM(64, input_shape=(training_data_x.shape[1:]), return_sequences=True),
    #Dropout(0.3),
    LSTM(64),
    #Dropout(0.3),
    Dense(1, activation='sigmoid')
])

LSTMmodel.compile(
    loss=BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.000003),
    metrics=["accuracy"]
)

log_dir = "logs/LSTM/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

csv_logger = CSVLogger('keras-models/KerasLSTM/training.csv',separator=",", append=False)

LSTMmodel.fit(training_data_x, training_data_y, epochs=10, validation_data=(validation_data_x,validation_data_y), callbacks=[tensorboard_callback, csv_logger])

model = LSTMmodel
model.save('keras-models/KerasLSTM')

Epoch 1/10
394/394 [==============================] - 11s 19ms/step - loss: 0.6850 - accuracy: 0.5816 - val_loss: 0.6775 - val_accuracy: 0.6976
Epoch 2/10
394/394 [==============================] - 6s 15ms/step - loss: 0.6702 - accuracy: 0.7580 - val_loss: 0.6612 - val_accuracy: 0.7706
Epoch 3/10
394/394 [==============================] - 6s 15ms/step - loss: 0.6519 - accuracy: 0.7708 - val_loss: 0.6397 - val_accuracy: 0.7706
Epoch 4/10
394/394 [==============================] - 6s 15ms/step - loss: 0.6269 - accuracy: 0.7708 - val_loss: 0.6094 - val_accuracy: 0.7706
Epoch 5/10
394/394 [==============================] - 6s 15ms/step - loss: 0.5915 - accuracy: 0.7708 - val_loss: 0.5665 - val_accuracy: 0.7706
Epoch 6/10
394/394 [==============================] - 6s 15ms/step - loss: 0.5436 - accuracy: 0.7708 - val_loss: 0.5125 - val_accuracy: 0.7706
Epoch 7/10
394/394 [==============================] - 6s 15ms/step - loss: 0.4927 - accuracy: 0.7708 - val_loss: 0.4689 - val_accuracy: 0.770

INFO:tensorflow:Assets written to: keras-models/KerasLSTM/assets


INFO:tensorflow:Assets written to: keras-models/KerasLSTM/assets


In [7]:
%tensorboard --logdir logs/LSTM/fit